Общее пояснение: данный файл содержит результаты ручной разметки текстов; одна строка = результаты работы одного ассесора по одному тексту; тексты повторяются, но их оценки уникальны. Конечная задача: обучить классификатор (в тесте не ставится)
Вкладка «куда вставлять» содержит файл, в котором неправильно заполнены столбцы с X по AD (отношение … опасный). Вкладка «что вставлять» содержит правильные значения этих столбцов. Неправильные значения надо заменить на правильные. Однако есть ряд проблем.

### Проблема 1.
Каждая строка вкладки "куда вставлять" имеет идентификационный номер, но номера повторяются. Если они повторяются, строки различаются по столбцу «логин». Во вкладке с правильными значениями столбцов X-AD ("что вставлять") сохранены IDs, но не сохранены значения столбца «логин». Таким образом, соотнести однозначно строки вкладки «что вставлять» и сторки вкладки «куда вставлять» невозможно. Надо брать ID из вкладки «что вставлять», искать его первое попавшееся упоминание во вкладке «куда вставлять» и, собственно, вставлять. Затем нужно взять во вкладке «что вставлять» следующую строку  с тем же ID и найти во вкладке «куда вставлять» СЛЕДУЮЩЕЕ (но не то же самое!) упоминание этого ID. Важно не запихать второй раз правильные значения в ту строку вкладки «куда вставлять», которую уже один раз заменили. А то в одной строке значения поменяем дважды, а в другой – ни разу.


### Проблема 2. 
Во вкладке «что вставлять» явно есть лишние данные, т.к. там почти в два раза больше строк. Те строки из вкладки «что вставлять», которые окажутся не востребованными и никуда не вставятся, записать в отдельный файл.


### Проблема 3. 
не исключено, если во вкладке «что вставлять» будет не хватать строк, которые есть во вкладке «что вставлять». Такой нехватки может и не быть. Но если все же во вкладке «куда вставлять» останутся строки, в которых ни разу не были заменены столбцы X-AD, такие строки надо сохранить в третий файл.


### NB:
обратите внимание, во вкладке "куда вставлять" столбцы не имеют уникальных заголовков. Так, заголовки "отношение", "опасный" и т.д. повторяются. Нас интересуют именно столбцы X-AD. В csv  и других форматах столбцы не будут иметь идентификаторов  типа X и AD, поэтому важно не перепутать между собой столбцы с одинаковыми заголовками. Нас интересует то "отношение", которые самое первое (самый левый из всех столбцов с заголовком "отношение"). Аналогично для других пяти столбцов.
результат представить в формате xls или csv (разделители целой части числа - запятые). Приложить код.


In [116]:
import pandas as pd
import numpy as np

In [159]:
where_all = pd.read_csv('тест датамйнерам.csv', low_memory=False)

In [118]:
#маска определяющаяя где у нас вообще есть id
id_mask = np.logical_not(where_all.id.isnull().values)
print id_mask.sum()


# выделем осмысленную часть датасета
where = where_all.iloc[id_mask]

2029


In [119]:
what = pd.read_csv('тест датамйнерам что.csv', low_memory=False)

Будем ходить двумя указателями, так что нужно, чтобы массивы были отсортированы по индексу. Порядок сортировки довольно специфический, так что функция сравнения своя

In [120]:
def id_compare(id_1, id_2):
    if id_1.isdigit() and id_2.isdigit():
        return int(id_1) <= int(id_2)
    elif id_1.isdigit():
        return True
    elif id_2.isdigit():
        return False
    else:
        return id_1 <= id_2

In [121]:
print all(map(lambda x : id_compare(x[0], x[1]), zip(what.id.values[:-1], what.id.values[1:])))
print all(map(lambda x : id_compare(x[0], x[1]), zip(where.id.values[:-1], where.id.values[1:]) ))

True
True


Оба датасета действительно отсортированы, дальше идем двумя указателями, что куда надо вставляем

In [122]:
i = 0
j = 0

unreplaced_where = pd.DataFrame(columns=where.columns)
unused_what = pd.DataFrame(columns=what.columns)

i_lim = where.shape[0]
j_lim = what.shape[0]

while i < i_lim and j < j_lim: 
    if i % 100 == 0:
        print 'i', i
        #print "j", j
    if where.loc[i, 'id'] == what.loc[j, 'id']:
        where.loc[i, what.columns] = what.iloc[j]
        i += 1
        j += 1
    elif id_compare(where.loc[i, 'id'], what.loc[j, 'id']):
        unreplaced_where.loc[i, where.columns] = where.iloc[i]
        i += 1   
    else:
        unused_what.loc[j, what.columns] = what.iloc[j]
        j += 1


i 0
i 0
i 100
i 200
i 300
i 400
i 400
i 500
i 500
i 500
i 600
i 700
i 800
i 900
i 900
i 900
i 1000
i 1000
i 1100
i 1200
i 1200
i 1300
i 1400
i 1500
i 1500
i 1600
i 1700
i 1700
i 1700
i 1700
i 1800
i 1800
i 1800
i 1800
i 1800
i 1800
i 1800
i 1800
i 1900
i 1900
i 1900
i 2000
i 2000
i 2000


Теперь проверим, дошли ли до конца, или остались строки, которые нужно куда-то сложить

In [124]:
print i, j
print i_lim, j_lim

2029 3344
2029 3344


Все на месте, кроме кусков из "куда вставлять", без id

Посмотрим глазами на то что получилось и сохраним 

In [186]:
where.iloc[1000:1004][what.columns]

,id,Обобщение,Отношение,Агрессия,Солидар.,Низ/выс,Жертва?,Опасный?
1000,3314000,2,2,1,1,3,3,1
1001,3314357,2,2,1,1,3,3,1
1002,3317701,2,2,1,1,3,3,1
1003,3318076,2,3,1,1,3,3,1


In [187]:
what.iloc[1270:1284]

,id,Обобщение,Отношение,Агрессия,Солидар.,Низ/выс,Жертва?,Опасный?
1270,3295491,2,2,1,1,3,3,1
1271,3295491,2,2,1,2,3,1,1
1272,3295491,2,3,1,2,3,2,2
1273,3297060,2,3,1,1,3,3,1
1274,3299549,3,2,1,1,3,3,1
1275,3299549,1,3,1,1,3,3,1
1276,3306375,2,3,1,1,3,3,1
1277,3312051,2,2,1,1,3,3,1
1278,3312051,2,2,1,2,3,3,1
1279,3314000,2,2,1,1,3,3,1


In [130]:
unused_what.reset_index(inplace = True)
unused_what.head()

,index,id,Обобщение,Отношение,Агрессия,Солидар.,Низ/выс,Жертва?,Опасный?
0,0,2942,3,2,1,1,3,3,1
1,12,55368,2,2,1,1,3,3,1
2,16,58073,2,2,1,1,3,3,1
3,21,62855,1,2,1,2,3,3,1
4,22,62855,1,2,1,1,3,3,1


In [131]:
unreplaced_where

,id,Время,Этноним,Логин,Понятно?,Этноним.1,Негатив,Позитив,Конфликт,Поз.взаим.,...,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83


In [158]:
what.to_csv('what.csv')
where.to_csv('where.csv', float_format='%.0f')
unused_what.to_csv('unused_what.csv')

In [172]:
nn_mask = np.logical_not(where_all.iloc[where_all.id.isnull().values].isnull()).values

In [175]:
pd.DataFrame(where_all.iloc[where_all.id.isnull().values].values[nn_mask]).to_csv('rubish.csv')

___________________
Дальше кусок отладки

In [89]:
where_val = np.array([['1', '2', 'as_111', 'as_12'], [0,0,0,0], [23,6,732,1]]).transpose()
what_val = np.array([['1', '2', '3', 'as_1111', 'as_12', 'as_322'], [1,2,3,4,5,6]]).transpose()

In [90]:
where = pd.DataFrame(where_val, columns=['id', 'val', 'bullshit'])
what = pd.DataFrame(what_val, columns=['id', 'val'])

In [94]:
where

,id,val,bullshit
0,1,1,23
1,2,2,6
2,as_111,0,732
3,as_12,5,1


In [113]:
what

,id,val
0,1,1
1,2,2
2,3,3
3,as_1111,4
4,as_12,5
5,as_322,6


In [101]:
for jj in range(j, what.shape[0]):
    unused_what.loc[j:\, what.columns] = what.iloc[j:]

In [108]:
unused_what

,id,val
2,3,3
3,as_1111,4


In [103]:
what.iloc[j:]

,id,val
5,as_322,6


In [188]:
what

,id,Обобщение,Отношение,Агрессия,Солидар.,Низ/выс,Жертва?,Опасный?
0,2942,3,2,1,1,3,3,1
1,9333,2,1,1,3,3,3,1
2,9333,3,1,1,3,3,3,1
3,12039,1,3,1,1,3,2,2
4,17208,2,2,1,2,3,3,1
5,19367,2,3,1,1,3,2,2
6,19367,2,3,1,1,1,2,2
7,21635,2,3,1,1,3,3,1
8,21635,1,2,1,1,3,3,2
9,22019,2,2,1,2,3,3,1


In [189]:
where

,id,Время,Этноним,Логин,Понятно?,Этноним.1,Негатив,Позитив,Конфликт,Поз.взаим.,...,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83
0,9333,10.07.2016 22:58,калмычка,Dasha,yes,one,no,weak,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9333,10.07.2016 22:58,калмычка,Dasha,yes,one,no,weak,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12039,27.07.2016 14:37,кавказец,Oleg Nagornyy,yes,one,weak,no,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17208,28.07.2016 18:28,сибиряк,Dasha,yes,one,no,weak,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19367,29.07.2016 15:04,жид,yadviga,yes,one,strong,no,yes,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,19367,29.07.2016 17:01,жид,Dasha,yes,one,weak,no,unk,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,21635,25.07.2016 16:27,горец,yadviga,yes,one,weak,weak,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,21635,03.08.2016 15:03,горец,Tatiana,yes,one,no,no,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,22019,15.07.2016 19:05,великоросс,Dasha,yes,one,no,no,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,50023,28.08.2016 0:56,калмык,Tatiana,yes,one,no,no,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
